# Increasing the Maximum Message Size for gRPC


## Running this notebook

You will need to start Jupyter with settings to allow for large payloads, for example:

```
jupyter notebook --NotebookApp.iopub_data_rate_limit=1000000000
```

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!pygmentize resources/model_long_timeouts.json

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [ ]:
!kubectl apply -f resources/model_long_timeouts.json -n seldon

In [ ]:
!kubectl rollout status deploy/model-long-timeout-test-0

## Get predictions

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="model-long-timeout",namespace="seldon", 
                  grpc_max_send_message_length=50 * 1024 * 1024, grpc_max_receive_message_length=50 * 1024 * 1024)

Send a small request which should suceed.

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc")
assert(r.success==True)
print(r)

Send a large request which will fail as the default for the model will be 4G.

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1000000,1))
print(r.success,r.msg)

In [ ]:
!kubectl delete -f resources/model_long_timeouts.json

## Allowing larger gRPC messages

Now we change our SeldonDeployment to include a annotation for max grpx message size.

In [ ]:
!pygmentize resources/model_grpc_size.json

In [ ]:
!kubectl create -f resources/model_grpc_size.json -n seldon

In [ ]:
!kubectl rollout status deploy/seldon-model-grpc-size-0 

Send a request via ambassador. This should succeed.

In [ ]:
sc = SeldonClient(deployment_name="seldon-model",namespace="seldon",
                  grpc_max_send_message_length=50 * 1024 * 1024, grpc_max_receive_message_length=50 * 1024 * 1024)
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1000000,1))
assert(r.success==True)
print(r.success)

In [ ]:
!kubectl delete -f resources/model_grpc_size.json -n seldon